In [ ]:
import numpy as np

import networkx as nx
import matplotlib.pyplot as plt

import os.path as op
import sys
sys.path.append("../")

from src import regmod
from src import utils

## Synthetic model

In [ ]:
path_to_data = "../resources"

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

filename = "adjacency_synthetic.pkl"

adj = utils.load(op.join(path_to_data, filename))
adj -= np.diag(np.diag(adj))

axes[0].imshow(adj, cmap='gray')
axes[0].set_title('Structural connectivity')
toy_graph = nx.Graph(adj)
nx.draw(toy_graph, ax=axes[1], with_labels=True)

In [ ]:
import importlib
importlib.reload(regmod)

max_path_depth = 3
alpha = 0.2

design_per_hops = regmod.get_path_matrices(adj, max_path_depth)
a_design_path = regmod.combine_paths_matrices(design_per_hops, alpha=alpha)

# Old degree designs
#a_design_degree_row = regmod.build_design_degree_fast(adj, target_deg=False)
#a_design_degree_col = regmod.build_design_degree_fast(adj, target_deg=True)

all_designs = [a_design_path] #, a_design_degree_row, a_design_degree_col]
all_titles = ["Paths"] #, "Degree Row", "Degree Col"]

fig, axes = plt.subplots(nrows=1, ncols=len(all_designs), figsize=(15, 5))
fig.suptitle("Design matrices")

for mat, axes, title in zip(all_designs, np.atleast_1d(axes), all_titles):
    axes.set_title(title + f", rank={np.linalg.matrix_rank(mat)}")
    axes.imshow(mat, cmap='gray')
    utils.add_cbar(fig, axes)

Zooming into the path desing matrix:

In [ ]:
fig, axes = plt.subplots(figsize=(10, 10))

axes.set_title("Design matrix (Paths)")
axes.imshow(a_design_path, cmap='gray', vmin=0, interpolation="none")

## Structural connectome (WM Bundles)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

filename = "adjacency_atlas.pkl"

adj_sc = utils.load(op.join(path_to_data, filename))
adj_sc -= np.diag(np.diag(adj_sc))

axes[0].imshow(adj_sc, cmap='gray')
axes[0].set_title('Structural connectivity')
toy_graph_sc = nx.Graph(adj_sc)
nx.draw(toy_graph_sc, ax=axes[1], with_labels=True)

In [ ]:
import importlib
importlib.reload(regmod)

max_path_depth = 2
alpha = 0.2

design_per_hops = regmod.get_path_matrices(adj_sc, max_path_depth)

a_design_sc_path = regmod.combine_paths_matrices(design_per_hops, alpha=alpha)

fig, axes = plt.subplots(figsize=(10, 10))

axes.set_title("Design matix (Paths)")
axes.imshow(a_design_sc_path, cmap='gray', vmin=0, interpolation="none")